In [13]:
%pip install tensorflow
%pip install matplotlib
%pip install pandas
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.2 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 4.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=6b8f3ebf8377fab3e2c4ce280731b231e02832347472d353c28d7cf92c6b034d
  Stored in directory: /home/fwendeburg/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-tr

# Loading the data

In [2]:
import glob

data_dir = 'data/generated'
data_files = glob.glob(data_dir + "/*.csv")

In [3]:
import pandas as pd

df_list = []

for file in data_files:
    data_frame = pd.read_csv(file, index_col=None, header=0)
    df_list.append(data_frame)

df_raw = pd.concat(df_list, ignore_index=True)

print(df_raw.shape)
print(df_raw.columns)

(995, 2)
Index(['pregunta', 'clase'], dtype='object')


In [4]:
df_test = pd.read_csv('data/preguntas_test.csv', index_col=None, header=0)
print(df_test.shape)
print(df_raw.columns)

(19, 2)
Index(['pregunta', 'clase'], dtype='object')


In [5]:
NUMBER_OF_CLASSES = df_raw['clase'].nunique()
MIN_NUMBER_OF_QUESTIONS = df_raw[df_raw['clase'] == df_raw.groupby('clase').size().idxmin()].shape[0]

In [6]:
df_raw = df_raw.groupby('clase').apply(lambda x: x.sample(n=MIN_NUMBER_OF_QUESTIONS)).reset_index(drop=True)
print(df_raw.shape)

(900, 2)


In [7]:
CLASSES_LIST = df_raw['clase'].unique().tolist()
print(CLASSES_LIST)

['Definicion', 'EntdidadGrupo', 'EntidadAbreviatura', 'EntidadLugar', 'EntidadLugarCapital', 'EntidadLugarCiudad', 'EntidadLugarPais', 'EntidadObjeto', 'EntidadPersona', 'Evento', 'NumericoCantidad', 'NumericoEdad', 'NumericoMedida', 'NumericoPeriodo', 'NumericoPorcentaje', 'TemporalAnyo', 'TemporalFecha', 'TemporalMes']


# Algorithm

In [86]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

In [84]:
def sample_rows(df, n):
    df_remaining = df[df['sampled'] == False]

    num_rows = len(df_remaining)
    if (n > num_rows):
        n = num_rows

    sampled_rows = df_remaining.sample(n)
    
    df.loc[sampled_rows.index, 'sampled'] = True

    return sampled_rows['pregunta'].tolist()

In [100]:
def classify_question(model, classes_list, questions_df, question, number_of_questions_to_sample):
    class_df = {}
    sampled_questions_values = {}
    for q_class in classes_list:
        class_df[q_class] = questions_df[questions_df['clase'] == q_class].drop('clase', axis=1)
        class_df[q_class]['sampled'] = False
        sampled_questions_values[q_class] = []

    # Assume that every class has the same number of example questions.
    number_of_questions = len(class_df[classes_list[0]])

    while number_of_questions > 0:
        for q_class in classes_list:
            questions = sample_rows(class_df[q_class], number_of_questions_to_sample)

            questions_embedding = model.encode(questions, convert_to_tensor=True)
            question_embedding = model.encode(question, convert_to_tensor=True)

            cos_scores = util.cos_sim(question_embedding, questions_embedding)[0]
            
            sampled_questions_values[q_class] = sampled_questions_values[q_class] + cos_scores.tolist()

            number_of_questions -= number_of_questions_to_sample
    
    avg_values_per_class = []
    for q_class in classes_list:
        avg_values_per_class.append(sum(sampled_questions_values[q_class])/len(class_df[classes_list[0]]))

    return classes_list[avg_values_per_class.index(max(avg_values_per_class))]

In [120]:
classify_question(model, CLASSES_LIST, df_raw, "¿Cuándo renunció Nixon?", 50)
        

'TemporalAnyo'